In [1]:
#import
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn import metrics

In [2]:
#Reading dataset
df=pd.read_csv('MF.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Columns: 150 entries, Cate_name to Yield
dtypes: int64(148), object(2)
memory usage: 79.8+ KB


In [3]:
#Building descriptors
X = df.drop(columns=['Yield', 'Cate_name', 'P_name'])
print('---Descriptors---')
print(X.head())

y = pd.DataFrame(df['Yield'],columns=['Yield'])
print('---Objective---')
print(y.head())

---Descriptors---
   Cate_MF_9  Cate_MF_33  Cate_MF_58  Cate_MF_68  Cate_MF_80  Cate_MF_102  \
0          0           0           0           0           0            0   
1          0           0           0           0           0            0   
2          0           0           0           0           0            0   
3          0           0           0           0           0            0   
4          0           0           0           0           0            0   

   Cate_MF_105  Cate_MF_114  Cate_MF_125  Cate_MF_142  ...  P_MF_1154  \
0            0            0            0            0  ...          1   
1            0            0            0            0  ...          0   
2            0            0            0            0  ...          0   
3            0            0            0            0  ...          0   
4            0            0            0            1  ...          1   

   P_MF_1199  P_MF_1272  P_MF_1380  P_MF_1570  P_MF_1689  P_MF_1750  \
0        

In [4]:
#storage 
data_r2_train =[]
data_RMSE_train = []
data_MAE_train = []
data_r2_test = []
data_RMSE_test = []
data_MAE_test = []
best_model_parametors = []
data_cv_score = []

#MLP
for i in range(10):
    seed=i
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    
    #autoscaling
    a_X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_test = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_train = a_X_train.dropna(how='any', axis=1)
    a_X_test = a_X_test[a_X_train.columns]
    a_y_train = y_train/10
    
    param_mlp={'hidden_layer_sizes':[(64,),(128,),(256,),(512,)], 'alpha':[0,2,4]}
    reg_mlp = GridSearchCV(MLPRegressor(random_state=0, max_iter=1000, learning_rate_init=0.03),
                           param_grid=param_mlp, cv=5, n_jobs=16)
    
    reg_mlp.fit(a_X_train,a_y_train['Yield'])
    reg_best = reg_mlp.best_estimator_
    y_pred1 = reg_best.predict(a_X_train)*10
    y_pred2 = reg_best.predict(a_X_test)*10
            
    #train
    r2_train = metrics.r2_score(y_train, y_pred1)
    RMSE_train = metrics.root_mean_squared_error(y_train, y_pred1)
    MAE_train =  metrics.mean_absolute_error(y_train, y_pred1)
    
    #test
    r2_test = metrics.r2_score(y_test, y_pred2)
    RMSE_test = metrics.root_mean_squared_error(y_test, y_pred2)
    MAE_test = metrics.mean_absolute_error(y_test, y_pred2)
    
    #strage data of accuracy 
    parametors = reg_mlp.best_params_
    best_model_parametors.append(parametors)

    data_r2_train.append(r2_train)
    data_RMSE_train.append(RMSE_train)
    data_MAE_train.append(MAE_train)
    data_r2_test.append(r2_test)
    data_RMSE_test.append(RMSE_test)
    data_MAE_test.append(MAE_test)
    data_cv_score.append(reg_mlp.best_score_)
    
    print('----------------------')
    print('seed:', seed)
    print("Best Model Parameter:",reg_mlp.best_params_)
    print("Best Model Score:",reg_mlp.best_score_)
    print('R2_test:', r2_test)
print('R2_train_means:', sum(data_r2_train)/10)
print('CV_score_means:', sum(data_cv_score)/10)
print('R2_test_means:', sum(data_r2_test)/10)

----------------------
seed: 0
Best Model Parameter: {'alpha': 0, 'hidden_layer_sizes': (128,)}
Best Model Score: 0.6655940719175915
R2_test: 0.6338523082918275
----------------------
seed: 1
Best Model Parameter: {'alpha': 4, 'hidden_layer_sizes': (512,)}
Best Model Score: 0.6293321185259376
R2_test: 0.7662910897754499
----------------------
seed: 2
Best Model Parameter: {'alpha': 2, 'hidden_layer_sizes': (512,)}
Best Model Score: 0.6168487919334438
R2_test: 0.7586662765340614
----------------------
seed: 3
Best Model Parameter: {'alpha': 4, 'hidden_layer_sizes': (256,)}
Best Model Score: 0.5891607616746108
R2_test: 0.7538906907415297
----------------------
seed: 4
Best Model Parameter: {'alpha': 0, 'hidden_layer_sizes': (128,)}
Best Model Score: 0.6409193485598271
R2_test: 0.6856789646212014
----------------------
seed: 5
Best Model Parameter: {'alpha': 4, 'hidden_layer_sizes': (512,)}
Best Model Score: 0.615316682723428
R2_test: 0.8379788061837624
----------------------
seed: 6
Best

In [5]:
data_r2_train_pd = pd.DataFrame(data= data_r2_train, columns=['r2_train'])
data_RMSE_train_pd = pd.DataFrame(data=data_RMSE_train, columns=['RMSE_train'])
data_MAE_train_pd = pd.DataFrame(data=data_MAE_train, columns=['MAE_train'])
data_r2_test_pd = pd.DataFrame(data=data_r2_test, columns=['r2_test'])
data_RMSE_test_pd = pd.DataFrame(data=data_RMSE_test, columns=['RMSE_test'])
data_MAE_test_pd = pd.DataFrame(data=data_MAE_test, columns=['MAE_test'])
data_cv_score_pd = pd.DataFrame(data=data_cv_score, columns=['cv_score'])
data_parametors = pd.DataFrame([best_model_parametors])

data_all = pd.concat([data_r2_train_pd, data_RMSE_train_pd, data_MAE_train_pd, data_r2_test_pd, 
                      data_RMSE_test_pd, data_MAE_test_pd, data_cv_score_pd],
                     axis=1, join='inner')

data_all.loc['mean'] = data_all.mean()
data_all.loc['std'] = data_all.std(ddof=0)
print(data_all)

data_all.to_csv('../../score/MF/MLP_score.csv')

      r2_train  RMSE_train  MAE_train   r2_test  RMSE_test   MAE_test  \
0     0.999964    0.163895   0.139147  0.633852  21.883851  15.973362   
1     0.986726    3.464617   2.399556  0.766291  15.047539  11.601248   
2     0.998312    1.249697   0.868692  0.758666  15.276927   9.592059   
3     0.995449    2.060178   1.351764  0.753891  15.368879  10.126610   
4     0.999968    0.166992   0.135795  0.685679  18.281827  14.908853   
5     0.991591    2.609368   1.958732  0.837979  14.037480   9.705857   
6     0.988269    3.102841   2.298037  0.800101  15.510338   9.699724   
7     0.995810    2.002041   1.308920  0.589845  19.282354  12.632451   
8     0.984453    3.699127   3.334814  0.520889  21.759473  16.416625   
9     0.996004    2.012469   1.815048  0.600054  16.787637  11.964318   
mean  0.993654    2.053122   1.561051  0.694725  17.323630  12.262111   
std   0.005051    1.123689   0.917261  0.094535   2.572438   2.402479   

      cv_score  
0     0.665594  
1     0.629332  